In [6]:
# scripts/genera_grafico_js.py
import os
import json
import numpy as np
import plotly.graph_objects as go

# 1) Genera datos y figura
rng = np.random.default_rng(seed=42)
x = np.arange(1, 31)
y = rng.normal(loc=100, scale=15, size=x.size).cumsum()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, y=y,
    mode="lines+markers",
    line=dict(color="#1e3a8a", width=2.5),
    marker=dict(size=6),
    name="Serie Aleatoria"
))

fig.update_layout(
    title="Serie temporal aleatoria (demo)",
    template="plotly_white",
    margin=dict(l=20, r=20, t=50, b=30),
    height=420
)

# 2) Serializa a JSON (figura)
fig_json = fig.to_plotly_json()
fig_json_str = json.dumps(fig_json, ensure_ascii=False)

# 3) Construye el código JS del módulo
js_code = f"""
// public/plots/grafico_demo.js
// Módulo UMD simple que expone renderPlot(containerId)
// y carga Plotly desde CDN si no está presente.

(function (root, factory) {{
  if (typeof define === 'function' && define.amd) {{
    define([], factory);
  }} else if (typeof module === 'object' && module.exports) {{
    module.exports = factory();
  }} else {{
    root.GraficoDemo = factory();
  }}
}}(typeof self !== 'undefined' ? self : this, function () {{

  const PLOTLY_CDN = "https://cdn.plot.ly/plotly-2.35.2.min.js";

  function ensurePlotly() {{
    return new Promise((resolve, reject) => {{
      if (window.Plotly) return resolve(window.Plotly);
      const script = document.createElement('script');
      script.src = PLOTLY_CDN;
      script.async = true;
      script.onload = () => resolve(window.Plotly);
      script.onerror = () => reject(new Error('No se pudo cargar Plotly desde CDN'));
      document.head.appendChild(script);
    }});
  }}

  // Figura en formato JSON exportada desde Python
  const fig = {fig_json_str};

  async function renderPlot(containerId) {{
    const Plotly = await ensurePlotly();
    const el = (typeof containerId === 'string') ? document.getElementById(containerId) : containerId;
    if (!el) throw new Error('Contenedor no encontrado: ' + containerId);

    const data = fig.data || [];
    const layout = fig.layout || {{}};
    const config = Object.assign({{}}, {{responsive: true, displaylogo: false}});

    // Estilo base del contenedor
    el.style.background = '#fff';
    el.style.borderRadius = '8px';

    await Plotly.newPlot(el, data, layout, config);
  }}

  return {{ renderPlot }};
}}));
"""

# 4) Escribe el archivo JS en public/plots/grafico_demo.js
here = os.path.dirname(os.path.abspath(__file__))       # .../scripts
project_root = os.path.abspath(os.path.join(here, ".."))# raíz del proyecto (landing_salud)
out_dir = os.path.join(project_root, "public", "plots")
os.makedirs(out_dir, exist_ok=True)
output_path = os.path.join(out_dir, "grafico_demo.js")

with open(output_path, "w", encoding="utf-8") as f:
  f.write(js_code)

print(f"Archivo JS generado en: {output_path}")

NameError: name '__file__' is not defined